# Demonstrate Seq2Seq Wrapper with twitter chat log

In [ ]:
import tensorflow as tf; print(tf.VERSION)

In [1]:
import tensorflow as tf
import numpy as np

# preprocessed data
from datasets.twitter import data
import data_utils

In [2]:
# load data from pickle and npy files
metadata, idx_q, idx_a = data.load_data(PATH='datasets/twitter/')
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_q, idx_a)

In [3]:
# parameters 
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 16
xvocab_size = len(metadata['idx2w'])  
yvocab_size = xvocab_size
emb_dim = 1024

In [4]:
import seq2seq_wrapper

In [5]:
import importlib
importlib.reload(seq2seq_wrapper)

<module 'seq2seq_wrapper' from '/Users/vladshostak/Desktop/tf-v1-venv/practical_seq2seq-master/seq2seq_wrapper.py'>

In [6]:
model = seq2seq_wrapper.Seq2Seq(xseq_len=xseq_len,
                               yseq_len=yseq_len,
                               xvocab_size=xvocab_size,
                               yvocab_size=yvocab_size,
                               ckpt_path='ckpt/twitter/',
                               emb_dim=emb_dim,
                               num_layers=3
                               )

<log> Started Building Graph... COMPLETE </log>

In [7]:
val_batch_gen = data_utils.rand_batch_gen(validX, validY, 256)
test_batch_gen = data_utils.rand_batch_gen(testX, testY, 256)
train_batch_gen = data_utils.rand_batch_gen(trainX, trainY, batch_size)

In [ ]:
#sess = model.train(train_batch_gen, val_batch_gen)

In [8]:
sess = model.restore_last_session()

In [22]:
input_ = test_batch_gen.__next__()[0]
output = model.predict(sess, input_)
print(output.shape)

[[   4   22  185 ...,    0    0    0]
 [  87 3175    5 ...,    0    0    0]
 [  13   22   21 ...,    0    0    0]
 ..., 
 [   4   22   21 ...,    0    0    0]
 [ 295    4   55 ...,    0    0    0]
 [  28   20    6 ...,    0    0    0]]


In [ ]:
replies = []
for ii, oi in zip(input_.T, output):
    q = data_utils.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded = data_utils.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split(' ')
    if decoded.count('unk') == 0:
        if decoded not in replies:
            print('q : [{0}]; a : [{1}]'.format(q, ' '.join(decoded)))
            replies.append(decoded)

In [9]:
# load data from pickle and npy files
idx2w, w2idx, limit = data_utils.get_metadata()

In [10]:
def respond(msg):
    encoded_msg = data_utils.encode(msg, w2idx, limit['maxq'])
    response = model.predict(sess, encoded_msg)[0]
    return data_utils.decode(response, idx2w)

In [29]:
respond('do you really')

'i think its a good idea'